# **Getting Started With Spark using Python**

![Spark Logo](http://spark.apache.org/images/spark-logo.png)

## How to run Jupyter notebook cell ?

Jupyter Notebooks are a powerful way to write and iterate on your Python code for data analysis.The Jupyter Notebook is organized into cells. Each cell can contain code, text, or visual elements. 
Rather than writing and re-writing an entire program, Jupyter Notebooks allow you to write code in separate blocks (or “cells”) and run each block of code individually. 
Then, if you need to make a change, you can go back and make an edit and rerun the program again, all in the same window.

To select the cell, simply click on it. The selected cell will be highlighted, indicating that it is ready for execution.Once you have the desired cell selected, you have multiple options to run it: 
* Keyboard Shortcut: Press Shift + Enter on your keyboard to run the selected cell. This will execute the code or process the content within the cell and display the output below it. 
If the cell has any output, it will be shown just after running.
* Toolbar: Look for the "Run Cell" button in the toolbar at the top of the Theia Lab interface. Clicking on this button will also execute the selected cell.
After running the cell, you can see the output displayed below it. If the cell contains any code that produces visual outputs, such as plots or images, they will be shown in the output area.

To run subsequent cells, repeat the same steps: select the next cell you want to execute and use the appropriate method mentioned above.
Remember to execute cells in the correct order if there are dependencies between them.
For example,if you have variables defined in a previous cell that are needed in the current cell, make sure to run the preceding cell first.


## Objectives

In this lab, we will go over the basics of Apache Spark and PySpark. We will start with creating the SparkContext and SparkSession. We then create a dataframe and demonstrate the basics dataframes and SparkSQL. Finally we create an RDD and apply some basic transformations and actions.

## Setup

In [64]:
!pip install pyspark
!pip install matplotlib
!pip install pandas
!pip install findspark
!pip install pyarrow

In [65]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql.functions import *
import pandas as pd
import matplotlib.pyplot as plt
import time

## Spark Context and Spark Session

You will create the Spark Context and initialize the Spark session needed for SparkSQL and DataFrames.
SparkContext is the entry point for Spark applications and contains functions to create RDDs such as `parallelize()`. SparkSession is needed for SparkSQL and DataFrame operations.


In [66]:
# Creating a spark context class
sc = SparkContext()

# # Creating a spark session
spark = SparkSession.builder.appName("pyspark-notebook").master("spark://spark-master:7077").config("spark.executor.memory", "1024m").getOrCreate()

In [67]:
spark

#### Initialize Spark session
To work with dataframes we just need to verify that the spark session instance has been created.

In [68]:
if 'spark' in locals() and isinstance(spark, SparkSession):
    print("SparkSession is active and ready to use.")
else:
    print("SparkSession is not active. Please create a SparkSession.")

SparkSession is active and ready to use.


## Load and Display dataset

In this section, you will first read the CSV file into a Pandas DataFrame and then read it into a Spark DataFrame.
Pandas is a library used for data manipulation and analysis. Pandas offers data structures and operations for creating and manipulating Data Series and DataFrame objects. Data can be imported from various data sources, e.g., Numpy arrays, Python dictionaries, and CSV files. Pandas allows you to manipulate, organize and display the data.

| colIndex | colName            | units/description                                                                             |
|----------|--------------------|-----------------------------------------------------------------------------------------------|
| [1]    | work_year          | The year in which the data was collected (2024).                                              |
| [2]    | experience_level   | The experience level of the employee, categorized as SE (Senior Engineer), MI (Mid-Level Engineer), or EL (Entry-Level Engineer). |
| [3]    | employment_type    | The type of employment, such as full-time (FT), part-time (PT), contract (C), or freelance (F).|
| [4]    | job_title          | The title or role of the employee within the company, for example, AI Engineer.               |
| [5]    | salary             | The salary of the employee in the local currency (e.g., 202,730 USD).                         |
| [6]    | salary_currency    | The currency in which the salary is denominated (e.g., USD).                                  |
| [7]    | salary_in_usd      | The salary converted to US dollars for standardization purposes (e.g., 202,730 USD).          |
| [8]    | employee_residence | The country of residence of the employee.                                                     |
| [9]    | remote_ratio       | The ratio indicating the extent of remote work allowed in the position (0 for no remote work, 100 for fully remote). |
| [10]    | company_location   | The location of the company where the employee is employed.                                   |
| [11]    | company_size       | The size of the company, often categorized by the number of employees (S for small, M for medium, L for large). |


In [69]:
# Read the file using `read_csv` function in pandas
file_path = './data/salaries.csv'

data_engineer_salary = pd.read_csv(file_path)


In [70]:
# Display only the first 10 rows with styles
styled_salaries = data_engineer_salary.head(10).style.set_properties(**{'text-align': 'left'}) 
styled_salaries.set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])

# Display the styled DataFrame
display(styled_salaries)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2024,SE,FT,AI Engineer,202730,USD,202730,US,0,US,M
1,2024,SE,FT,AI Engineer,92118,USD,92118,US,0,US,M
2,2024,SE,FT,Data Engineer,130500,USD,130500,US,0,US,M
3,2024,SE,FT,Data Engineer,96000,USD,96000,US,0,US,M
4,2024,SE,FT,Machine Learning Engineer,190000,USD,190000,US,0,US,M
5,2024,SE,FT,Machine Learning Engineer,160000,USD,160000,US,0,US,M
6,2024,MI,FT,ML Engineer,400000,USD,400000,US,0,US,M
7,2024,MI,FT,ML Engineer,65000,USD,65000,US,0,US,M
8,2024,EN,FT,Data Analyst,101520,USD,101520,US,0,US,M
9,2024,EN,FT,Data Analyst,45864,USD,45864,US,0,US,M


## DataFrames

A DataFrame is two-dimensional. Columns can be of different data types. DataFrames accept many data inputs including series and other DataFrames. You can pass indexes (row labels) and columns (column labels). Indexes can be numbers, dates, or strings/tuples.


### Load the data into Spark dataframe

In [71]:
schema = StructType([
    StructField("work_year", IntegerType(), True),
    StructField("experience_level", StringType(), True),
    StructField("employment_type", StringType(), True),
    StructField("job_title", StringType(), True),
    StructField("salary", IntegerType(), True),
    StructField("salary_currency", StringType(), True),
    StructField("salary_in_usd", IntegerType(), True),
    StructField("employee_residence", StringType(), True),
    StructField("remote_ratio", IntegerType(), True),
    StructField("company_location", StringType(), True),
    StructField("company_size", StringType(), True)
])

In [72]:
# Convert Pandas DataFrame to Spark DataFrame with defined schema
salaries_df = spark.createDataFrame(data_engineer_salary.values.tolist(), schema=schema)
# Show the DataFrame schema and some data
salaries_df.printSchema()

root
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



###  Basic data analysis and manipulation
In this section, we perform basic data analysis and manipulation. We start with previewing the data and then applying some filtering and columwise operations.

#### Task 1:

Show the first 5 records of the DataFrame.

In [73]:
# to-do
salaries_df.show(5)

+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|     2024|              SE|             FT|         AI Engineer|202730|            USD|       202730|                US|           0|              US|           M|
|     2024|              SE|             FT|         AI Engineer| 92118|            USD|        92118|                US|           0|              US|           M|
|     2024|              SE|             FT|       Data Engineer|130500|            USD|       130500|                US|           0|              US|           M|
|     2024

#### Task 2:
Select the `salary` and `job_title` columns from the DataFrame and display the first 5 rows of this columns.

In [74]:
# to-do
salaries_df.select('salary', 'job_title').show(5)

+------+--------------------+
|salary|           job_title|
+------+--------------------+
|202730|         AI Engineer|
| 92118|         AI Engineer|
|130500|       Data Engineer|
| 96000|       Data Engineer|
|190000|Machine Learning ...|
+------+--------------------+
only showing top 5 rows



#### Task 3: 

Display the first five rows of `salary` and `job_title` and `salary_currency` from the DataFrame where the salary is less than 50,000. 

In [75]:
# to-do
salaries_df.select('salary', 'job_title', 'salary_currency').filter(col('salary') < 50000).show(5)

+------+---------------+---------------+
|salary|      job_title|salary_currency|
+------+---------------+---------------+
| 45864|   Data Analyst|            USD|
| 45000| Data Architect|            GBP|
| 30000|   Data Analyst|            EUR|
| 25800|   Data Analyst|            EUR|
| 45607|Data Specialist|            GBP|
+------+---------------+---------------+
only showing top 5 rows



#### Task 4: 

Multiply the salary column by 0.5 and display the `salary`, `job_title`, and the newly calculated half_salary for the first five rows.

In [76]:
# to-do
salaries_df.withColumn('salary', col('salary') * 0.5).select('salary', 'job_title').show(5)

+--------+--------------------+
|  salary|           job_title|
+--------+--------------------+
|101365.0|         AI Engineer|
| 46059.0|         AI Engineer|
| 65250.0|       Data Engineer|
| 48000.0|       Data Engineer|
| 95000.0|Machine Learning ...|
+--------+--------------------+
only showing top 5 rows



#### Task 5: 

Join these two dataframes on `emp_id`.

In [77]:
# DataFrame 1 

data = [("A101", "John"), ("A102", "Peter"), ("A103", "Charlie")] 

columns = ["emp_id", "emp_name"] 

dataframe_1 = spark.createDataFrame(data, columns) 

In [78]:
# DataFrame 2 

data = [("A101", 1000), ("A102", 2000), ("A103", 3000)]

columns = ["emp_id", "salary"]

dataframe_2 = spark.createDataFrame(data, columns)

In [79]:
# to-do 
# create a new DataFrame, "combined_df" by performing an inner join
combined_df = dataframe_1.join(dataframe_2, dataframe_1.emp_id == dataframe_2.emp_id, 'inner')
combined_df.show()

+------+--------+------+------+
|emp_id|emp_name|emp_id|salary|
+------+--------+------+------+
|  A103| Charlie|  A103|  3000|
|  A102|   Peter|  A102|  2000|
|  A101|    John|  A101|  1000|
+------+--------+------+------+



#### Task6:

Filter the DataFrame to include only entries with experience levels "SE" (Senior Engineer) and "MI" (Mid-Level Engineer), then calculate and display the average salary for each experience level.

In [80]:
# to-do 
filtered_df = salaries_df.filter((col('experience_level') == 'SE') | (col('experience_level') == 'MI'))
filtered_df.groupBy(col('experience_level')).agg(avg('salary').alias('average_salary')).show()

+----------------+------------------+
|experience_level|    average_salary|
+----------------+------------------+
|              MI|157116.97672114908|
|              SE|168852.49343955013|
+----------------+------------------+



#### Task7
Write datadrame in the `HDFS` and load

In [81]:
# to-do 
salaries_df.write.format('csv').mode('overwrite').save('target/', header='true') #writing

loaded_df = spark.read.csv('target/', inferSchema=True, header='true') #loading
loaded_df.show()

+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|     2023|              MI|             FT|       Data Engineer|175000|            USD|       175000|                US|           0|              US|           M|
|     2023|              SE|             FT|      Data Scientist|136000|            USD|       136000|                US|         100|              US|           M|
|     2023|              SE|             FT|      Data Scientist|104000|            USD|       104000|                US|         100|              US|           M|
|     2023

# **Introduction to SparkSQL**

Spark SQL is a Spark module for structured data processing. It is used to query structured data inside Spark programs, using either SQL or a familiar DataFrame API.

####  Create a Table View
Creating a table view in Spark SQL is required to run SQL queries programmatically on a DataFrame. A view is a temporary table to run SQL queries. A Temporary view provides local scope within the current Spark session. In this example we create a temporary view using the `createTempView()` function. Once we have a table view, we can run queries similar to querying a SQL table.


In [82]:
salaries_df.createTempView("salaries")

####  Running SQL queries and aggregating data

####  Task 8:
Display the first five records of AI Engineers with salaries over 10,000 with Spark SQL.

In [83]:
# to-do
spark.sql("SELECT * \
          FROM salaries \
          WHERE job_title = 'AI Engineer' \
          AND salary > 10000;").show(5)

+---------+----------------+---------------+-----------+------+---------------+-------------+------------------+------------+----------------+------------+
|work_year|experience_level|employment_type|  job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---------+----------------+---------------+-----------+------+---------------+-------------+------------------+------------+----------------+------------+
|     2024|              SE|             FT|AI Engineer|202730|            USD|       202730|                US|           0|              US|           M|
|     2024|              SE|             FT|AI Engineer| 92118|            USD|        92118|                US|           0|              US|           M|
|     2024|              MI|             FT|AI Engineer|150000|            USD|       150000|                US|         100|              US|           M|
|     2024|              MI|             FT|AI Engineer| 90200| 

#### Task 9:
Execute a SQL query to find and display the maximum salaries for job titles in `large` `US` companies, then plot the top 10 highest salaries using a horizontal bar chart.

In [84]:
# to-do
spark.sql("SELECT job_title, max(salary_in_usd) as max_salary_in_usd \
          FROM salaries \
          WHERE company_location = 'US' \
          AND company_size = 'L' \
          GROUP BY job_title \
          ORDER BY max_salary_in_usd DESC;").show()

+--------------------+-----------------+
|           job_title|max_salary_in_usd|
+--------------------+-----------------+
|Applied Machine L...|           423000|
|      Data Scientist|           412000|
| Data Analytics Lead|           405000|
|  Research Scientist|           405000|
|Applied Data Scie...|           380000|
|Data Science Tech...|           375000|
|   Applied Scientist|           350000|
|Machine Learning ...|           342300|
|Director of Data ...|           325000|
|        Data Science|           324100|
|Managing Director...|           300000|
|        Head of Data|           290000|
|        AI Architect|           289000|
|         ML Engineer|           287700|
|  Lead Data Engineer|           276000|
|Data Science Manager|           272400|
|   Research Engineer|           261500|
|  AWS Data Architect|           258000|
|Data Analytics Ma...|           254500|
|Cloud Data Architect|           250000|
+--------------------+-----------------+
only showing top

#### Task 10:
Write a Spark SQL query to count the instances where a Data Engineer's salary in USD is greater than that of a Data Analyst, considering matching experience level, employment type, and company location.

In [85]:
# to do
spark.sql("SELECT count(*) \
          FROM salaries S1 join salaries S2 \
          USING (experience_level, employment_type, company_location) \
          WHERE S1.job_title = 'Data Engineer' \
          AND S2.job_title = 'Data Analyst' \
          AND S1.salary_in_usd > S2.salary_in_usd;").show()

+--------+
|count(1)|
+--------+
| 2060983|
+--------+



#### Create an RDD

In [86]:
# Convert DataFrame to RDD
rdd = salaries_df.rdd

#### Transformations

A transformation is an operation on an RDD that results in a new RDD. The transformed RDD is generated rapidly because the new RDD is lazily evaluated, which means that the calculation is not carried out when the new RDD is generated. The RDD will contain a series of transformations, or computation instructions, that will only be carried out when an action is called. Note the use of the lambda function.

In [87]:
high_salary_rdd = rdd.filter(lambda row: row['salary'] > 200000)

#### Actions 

A transformation returns a result to the driver. We now apply the `count()` action to get the output from the transformation.

In [88]:
high_salary_rdd.count()

3262

#### Task 11:
Create an RDD with integers from 1-50. Apply a transformation to multiply every number by 2, resulting in an RDD that contains the first 50 even numbers. 



In [89]:
# to do
rdd = sc.parallelize(range(1, 50))
even_numbers = rdd.map(lambda number: number * 2)
results = even_numbers.collect()
print(results)

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98]


#### Task 12:
Filter and collect all records from an RDD for employees with the job title "Data Science Lead"

In [90]:
# to do
rdd = salaries_df.rdd
filtered_rdd = rdd.filter(lambda row: row['job_title'] == 'Data Science Lead')
filtered_rdd.collect()

[Row(work_year=2024, experience_level='SE', employment_type='FT', job_title='Data Science Lead', salary=185000, salary_currency='USD', salary_in_usd=185000, employee_residence='US', remote_ratio=0, company_location='US', company_size='M'),
 Row(work_year=2024, experience_level='SE', employment_type='FT', job_title='Data Science Lead', salary=120000, salary_currency='USD', salary_in_usd=120000, employee_residence='US', remote_ratio=0, company_location='US', company_size='M'),
 Row(work_year=2023, experience_level='SE', employment_type='FT', job_title='Data Science Lead', salary=148300, salary_currency='USD', salary_in_usd=148300, employee_residence='US', remote_ratio=0, company_location='US', company_size='M'),
 Row(work_year=2023, experience_level='SE', employment_type='FT', job_title='Data Science Lead', salary=84700, salary_currency='USD', salary_in_usd=84700, employee_residence='US', remote_ratio=0, company_location='US', company_size='M'),
 Row(work_year=2023, experience_level='SE'

#### Task 13:
Calculate the total `salary` paid by `large` companies by filtering and aggregating salary data in an RDD using Apache Spark.

In [91]:
# to do
rdd = salaries_df.rdd
filtered_rdd = rdd.filter(lambda row: row['company_size'] == 'L')
sum_salaries = filtered_rdd.aggregate(0,
                                      lambda acc, row: acc + row['salary'],
                                      lambda acc1, acc2: acc1 + acc2)
sum_salaries

320740143

#### Task 14:
Filter and display the top five highest-paying fully remote job titles based on salary.

In [92]:
# to do
rdd = salaries_df.rdd
remote_rdd = rdd.filter(lambda row: row['remote_ratio'] == 100)
sorted_rdd = remote_rdd.sortBy(lambda row: row['salary'], ascending=False)
for item in sorted_rdd.take(5):
    print(f"Job Title: {item['job_title']}, Salary: {item['salary']}")

Job Title: Data Scientist, Salary: 30400000
Job Title: Data Scientist, Salary: 6600000
Job Title: Data Engineer, Salary: 4450000
Job Title: Data Scientist, Salary: 4200000
Job Title: Data Scientist, Salary: 4000000


##  Create a Pandas UDF
Apache Spark has become the de-facto standard in processing big data. To enable data scientists to leverage the value of big data, Spark added a Python API in version 0.7, with support for user-defined functions (UDF). These user-defined functions operate one-row-at-a-time, and thus suffer from high serialization and invocation overhead. As a result, many data pipelines define UDFs in Java and Scala and then invoke them from Python.

Pandas UDFs built on top of Apache Arrow bring you the ability to define low-overhead, high-performance UDFs entirely in Python.

In addition, UDFs can be registered and invoked in SQL out of the box by registering a regular python function using the `@pandas_udf()` decorator.  

[pyspark.sql.functions.udf](https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html)

[pyspark.sql.DataFrame.withColumn](https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.DataFrame.withColumn.html)


#### Task15:
Implement a Pandas UDF in PySpark named `categorize_salary` that accepts a pandas Series containing salaries and categorizes each salary into "Low" (less than $50,000), "Medium" ($50,000 to $149,999), or "High" ($150,000 and above) by returning a new Series with these categories.

In [93]:
# to do
@pandas_udf(StringType())
def categorize_salary(salary_series: pd.Series) -> pd.Series:
    categories = []
    for salary in salary_series:
        if salary < 50000:
            categories.append("Low")
        elif 50000 <= salary < 150000:
            categories.append("Medium")
        else:
            categories.append("High")
    return pd.Series(categories)

#### Task 16:
Integrate the `categorize_salary` UDF into a PySpark DataFrame by adding a new column salary_category which classifies each salary. Subsequently, display the original salary and its corresponding category to verify the successful application of the UDF.

In [94]:
# to do
classified_df = salaries_df.withColumn("salary_category", categorize_salary(col("salary")))
classified_df.select("salary", "salary_category").show()

+------+---------------+
|salary|salary_category|
+------+---------------+
|202730|           High|
| 92118|         Medium|
|130500|         Medium|
| 96000|         Medium|
|190000|           High|
|160000|           High|
|400000|           High|
| 65000|         Medium|
|101520|         Medium|
| 45864|            Low|
|172469|           High|
|114945|         Medium|
|200000|           High|
|150000|           High|
|156450|           High|
|119200|         Medium|
|170000|           High|
|130000|         Medium|
|222200|           High|
|136000|         Medium|
+------+---------------+
only showing top 20 rows



#### Bonus 1:
1.Calculating sum of salary with company size 'L' without caching.

In [95]:
rdd = salaries_df.rdd

start_time_no_cache = time.time()
dummy = rdd.filter(lambda row: row['company_size'] == 'L').aggregate(0,
                                                                     lambda acc, row: acc + row['salary'],
                                                                     lambda acc1, acc2: acc1 + acc2)
end_time_no_cache = time.time()
end_time_no_cache - start_time_no_cache

0.2176377773284912

2.Calculating sum of salary with company size 'L' with caching.

In [96]:
rdd_cached = rdd.cache()

start_time_cache = time.time()
dummy = rdd_cached.filter(lambda row: row['company_size'] == 'L').aggregate(0,
                                                                            lambda acc, row: acc + row['salary'],
                                                                            lambda acc1, acc2: acc1 + acc2)
end_time_cache = time.time()
end_time_cache - start_time_cache

0.3096776008605957

#### Bonus 2:
1.Narrow transforamtion example.

In [97]:
rdd = sc.parallelize(range(1, 1000000))

start_time_narrow = time.time()
narrow_result = rdd.map(lambda x: x * 2).collect()
end_time_narrow = time.time()
end_time_narrow - start_time_narrow

0.42276501655578613

2.Wide transforamtion example.

In [98]:
rdd_map = rdd.map(lambda x: (x % 100, x))

start_time_wide = time.time()
wide_result = rdd_map.groupByKey().mapValues(list).collect()
end_time_wide = time.time()
end_time_wide - start_time_wide

0.953167200088501

In [63]:
spark.stop()

## Authors
[Mohammad Rahmanian](https://github.com/Mohammad-Rahmanian)

### Other Contributors
[Mohammad Sadegh Mohammadi](https://github.com/sadegh-msm)